## Finding fusions - Angus's code + my adaptation for soft clips which do NOT have an hg38 map

In [1]:
#packages
import os
import pysam
import re
import pandas as pd

In [4]:
# Define patient configurations with detailed Breakpoint_IDs
patients_info = [
    {
        "PatientID": "patient_FT",
        "BAMs": [
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449539/possorted_genome_bam39.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449540/possorted_genome_bam40.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449541/possorted_genome_bam41.bam"
        ],
        "Breakpoints": [
            {"chr1": "19", "pos1": 42295139, "chr2": "4", "pos2": 190174732, "gene1": "CIC", "gene2": "DUX4", "Breakpoint_ID": "CIC_42295139_DUX4_190174732"}
        ]
    },
    {
        "PatientID": "patient_FO",
        "BAMs": [
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960948/possorted_genome_bam48.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960949/possorted_genome_bam49.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960950/possorted_genome_bam50.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/possorted_genome_bam51.bam"
        ],
        "Breakpoints": [
            {"chr1": "19", "pos1": 42295139, "chr2": "4", "pos2": 190174732, "gene1": "CIC", "gene2": "DUX4", "Breakpoint_ID": "CIC_42295139_DUX4_190174732"}
        ]
    }
]

output_dir = "/lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/"

In [17]:
# Ensure output directory exists
output_dir = "/lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/"
os.makedirs(output_dir, exist_ok=True)

# Define CIGAR regex pattern
cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')
wiggle_room = 50  # Allow 50 bp wiggle room around breakpoints

# Function to extract read metrics, including handling soft-clipped portions
def extract_read_metrics(read, bam_prefix):
    read_status = "Unmapped" if read.is_unmapped else ("Multi-mapped" if read.has_tag("NH") and read.get_tag("NH") > 1 else "Mapped")
    num_alignments = read.get_tag("NH") if read.has_tag("NH") else None
    alignment_score = read.get_tag("AS") if read.has_tag("AS") else None
    read_sequence = read.query_sequence if read.query_sequence else "No sequence"

    # Extract Cell Barcode (CB tag) or fallback to CR tag
    cell_barcode = dict(read.get_tags()).get("CB") or dict(read.get_tags()).get("CR")
    cell_barcode = cell_barcode.replace("-1-1", "-1") if cell_barcode else "Unknown"
    cell_id = f"{bam_prefix}::{cell_barcode}" if cell_barcode != "Unknown" else "Unknown"

    # Handle soft-clipped portions and determine last mapped position
    cigar_string = read.cigarstring if read.cigarstring else ""
    soft_clipped_start = ""
    soft_clipped_end = ""
    start_position = read.reference_start
    end_position = read.reference_end

    # Initialize positions for soft clips
    soft_clip_start_pos = None
    soft_clip_end_pos = None

    # Parse CIGAR string for soft-clipped portions
    parsed_cigar = cigar_pattern.findall(cigar_string)
    current_pos = read.reference_start

    for length, operation in parsed_cigar:
        length = int(length)
        if operation == "S":
            if current_pos == read.reference_start:
                # Soft-clipped at the start
                soft_clipped_start = read_sequence[:length]
                soft_clip_start_pos = current_pos - length  # Position before the start of the alignment
            else:
                # Soft-clipped at the end
                soft_clipped_end = read_sequence[-length:]
                soft_clip_end_pos = current_pos  # Position at the end of the alignment
        elif operation in ("M", "D", "N", "="):
            current_pos += length

    # Update last mapped position if the read ends with soft-clipping
    if soft_clipped_end:
        end_position = current_pos - 1

    # Full read sequence including soft-clipped portions
    full_read_sequence = f"{soft_clipped_start}{read_sequence}{soft_clipped_end}"

    # Return additional info with soft clip positions
    return read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, end_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos

# Main processing loop for each patient

for patient in patients_info:
    patient_id = patient["PatientID"]
    breakpoints = patient["Breakpoints"]

    for bam_path in patient["BAMs"]:
        bam_prefix = os.path.basename(os.path.dirname(bam_path))
        output_file = os.path.join(output_dir, f"{patient_id}_{bam_prefix}_reads_over_breakpoints.tsv")
        output_data = []

        # Open the BAM file
        with pysam.AlignmentFile(bam_path, "rb") as bam:
            for bp in breakpoints:
                chr1, pos1, chr2, pos2 = bp["chr1"], bp["pos1"], bp["chr2"], bp["pos2"]
                gene1, gene2 = bp["gene1"], bp["gene2"]

                # Define the Breakpoint_ID
                breakpoint_id = f"{gene1}_{pos1}_{gene2}_{pos2}"

                # Fetch reads overlapping the breakpoints
                for read in bam.fetch(chr1, pos1 - wiggle_room, pos1 + wiggle_room):
                    read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos = extract_read_metrics(read, bam_prefix)
                    output_data.append([ 
                        patient_id, bam_path, read.query_name, chr1, read.reference_start,
                        read.reference_end, read_status, num_alignments, alignment_score,
                        full_read_sequence, cell_barcode, cell_id, breakpoint_id,
                        last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos
                    ])

                for read in bam.fetch(chr2, pos2 - wiggle_room, pos2 + wiggle_room):
                    read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos = extract_read_metrics(read, bam_prefix)
                    output_data.append([ 
                        patient_id, bam_path, read.query_name, chr2, read.reference_start,
                        read.reference_end, read_status, num_alignments, alignment_score,
                        full_read_sequence, cell_barcode, cell_id, breakpoint_id,
                        last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos
                    ])

        # Define columns and save the extracted data
        columns = [
            "PatientID", "BAM", "ReadName", "Chromosome", "Start", "End",
            "ReadStatus", "NumAlignments", "AlignmentScore", "FullReadSequence",
            "CellBarcode", "CellID", "Breakpoint_ID", "LastMappedPosition",
            "SoftClippedStart", "SoftClippedEnd", "SoftClipStartPos", "SoftClipEndPos"
        ]
        df = pd.DataFrame(output_data, columns=columns)
        df.to_csv(output_file, sep='\t', index=False)
        print(f"Completed processing for patient {patient_id}. Output saved to: {output_file}")

print("All patient data processed successfully.")

Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449539_reads_over_breakpoints.tsv
Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449540_reads_over_breakpoints.tsv
Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449541_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FO_CG_SB_NB13960948_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FO_CG_SB_NB13960949_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_o

In [10]:
def split_sequence(row, breakpoints):
    read_sequence = row["FullReadSequence"]
    start_pos = row["Start"]
    end_pos = row["End"]
    chr_name = row["Chromosome"]
    breakpoint_id = row["Breakpoint_ID"]
    soft_clipped_start = row["SoftClippedStart"] if pd.notna(row["SoftClippedStart"]) else ""
    soft_clipped_end = row["SoftClippedEnd"] if pd.notna(row["SoftClippedEnd"]) else ""

    # Calculate chromosomal location of soft clips
    soft_clip_start_chromosomal_location = None
    soft_clip_end_chromosomal_location = None

    if soft_clipped_start:
        soft_clip_start_chromosomal_location = start_pos - len(soft_clipped_start)

    if soft_clipped_end:
        soft_clip_end_chromosomal_location = end_pos + len(soft_clipped_end)

    # Now split the sequence (no breakpoint logic)
    before_breakpoint = read_sequence[:start_pos] if start_pos > 0 else ""
    after_breakpoint = read_sequence[end_pos:] if end_pos < len(read_sequence) else ""

    # Incorporate soft-clipped sequences into their respective locations
    if soft_clipped_start:
        before_breakpoint = soft_clipped_start + before_breakpoint
    if soft_clipped_end:
        after_breakpoint = after_breakpoint + soft_clipped_end

    # Ensure the function always returns three values
    return before_breakpoint, after_breakpoint, f"ValidSplit: SoftClipStartChromosomalLocation={soft_clip_start_chromosomal_location}, SoftClipEndChromosomalLocation={soft_clip_end_chromosomal_location}"



# Annotate and split sequences for each patient
for patient in patients_info:
    patient_id = patient["PatientID"]
    breakpoints = patient["Breakpoints"]

    for bam_path in patient["BAMs"]:
        bam_prefix = os.path.basename(os.path.dirname(bam_path))
        input_file = os.path.join(output_dir, f"{patient_id}_{bam_prefix}_reads_over_breakpoints.tsv")
        output_file = os.path.join(annotated_output_dir, f"{patient_id}_{bam_prefix}_annotated_split_sequences.tsv")

        if not os.path.exists(input_file):
            print(f"Warning: Input file {input_file} not found. Skipping.")
            continue

        print(f"Processing {input_file}")
        
        # Load the file first
        df = pd.read_csv(input_file, sep='\t')
        print(f"Input rows: {len(df)}")

        # Apply your split function
        df[["BeforeBreakpoint", "AfterBreakpoint", "SplitStatus"]] = df.apply(
            split_sequence, axis=1, result_type="expand", breakpoints=breakpoints
        )

        # Show counts for debugging
        print("SplitStatus counts (all rows):")
        print(df["SplitStatus"].value_counts())

        # Keep only valid splits
        valid_df = df[df["SplitStatus"] == "ValidSplit"].drop(columns=["SplitStatus"])

        # Save results
        valid_df.to_csv(output_file, sep='\t', index=False)
        print(f"Annotated split sequences saved to: {output_file}")


Processing /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449539_reads_over_breakpoints.tsv
Input rows: 165
SplitStatus counts (all rows):
SplitStatus
ValidSplit: SoftClipStartChromosomalLocation=None, SoftClipEndChromosomalLocation=None        134
ValidSplit: SoftClipStartChromosomalLocation=None, SoftClipEndChromosomalLocation=42307731     11
ValidSplit: SoftClipStartChromosomalLocation=None, SoftClipEndChromosomalLocation=42446747      3
ValidSplit: SoftClipStartChromosomalLocation=None, SoftClipEndChromosomalLocation=42307765      2
ValidSplit: SoftClipStartChromosomalLocation=None, SoftClipEndChromosomalLocation=42295127      2
ValidSplit: SoftClipStartChromosomalLocation=42091050, SoftClipEndChromosomalLocation=None      1
ValidSplit: SoftClipStartChromosomalLocation=42141587, SoftClipEndChromosomalLocation=None      1
ValidSplit: SoftClipStartChromosomalLocation=42098255, SoftClipEndChromosomalLocation=None      1
ValidSplit: SoftClipStartChromo

In [ ]:
# File paths
metadata_csv = "/lustre/scratch126/casm/team274sb/project_folders/GOSH_Leuk/sample_metadata/Patient_profiles/AHLJ_annot_oct2024_overall.csv"
annotated_output_dir = os.path.join(output_dir, "annotated_split_sequences")

# Load metadata with 'CellID' and 'AHLJ_annot_oct2024' annotations
metadata_df = pd.read_csv(metadata_csv, sep=',')[["CellID", "AHLJ_annot_oct2024"]]

# Loop through all annotated split sequence files
for filename in os.listdir(annotated_output_dir):
    if filename.endswith("_annotated_split_sequences.tsv"):
        input_file = os.path.join(annotated_output_dir, filename)
        print(f"Processing file: {filename}")

        # Load the annotated split sequence file
        df = pd.read_csv(input_file, sep='\t')

        # Merge with metadata to include AHLJ annotations
        merged_df = pd.merge(df, metadata_df, on='CellID', how='left')

        # Save the merged DataFrame back to the same file
        merged_df.to_csv(input_file, sep='\t', index=False)
        print(f"Updated file saved with AHLJ annotations: {input_file}")

print("All files updated with AHLJ annotations successfully.")

In [ ]:
import os
import pandas as pd

# Define the directory containing annotated split sequence files
annotated_split_dir = os.path.join(output_dir, "annotated_split_sequences")

# List to store dataframes
dataframes = []

# Initialize a counter for UniqueID
unique_id_counter = 1

# Iterate over each file in the directory
for file_name in os.listdir(annotated_split_dir):
    if file_name.endswith("_annotated_split_sequences.tsv"):
        file_path = os.path.join(annotated_split_dir, file_name)
        # Load the annotated file
        df = pd.read_csv(file_path, sep='\t')

        # Add a UniqueID column for each row in this file
        num_rows = len(df)
        df["UniqueID"] = range(unique_id_counter, unique_id_counter + num_rows)

        # Increment the counter for the next file
        unique_id_counter += num_rows

        # Append the dataframe to the list
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dataframes, ignore_index=True)

# Define the output file path for the merged file
merged_output_file = os.path.join(output_dir, "all_patients_annotated_split_sequences_merged.tsv")

# Save the merged dataframe to a TSV file
merged_df.to_csv(merged_output_file, sep='\t', index=False)
print(f"All annotated split sequences merged and saved to: {merged_output_file}")

In [ ]:
# Input and output paths
merged_file = os.path.join(output_dir, "all_patients_annotated_split_sequences_merged.tsv")
fasta_output_file = os.path.join(output_dir, "prepared_blat_input.fasta")

# Load the merged file
df = pd.read_csv(merged_file, sep='\t')

# Define the columns to extract sequences from
sequence_columns = ["SoftClippedStart", "SoftClippedEnd", "BeforeBreakpoint", "AfterBreakpoint"]

# Filter sequences that are not empty and have at least 20 bases
sequences_to_blat = []

for _, row in df.iterrows():
    for column in sequence_columns:
        sequence = str(row[column])
        if pd.notna(sequence) and len(sequence) >= 20:
            unique_id = row["UniqueID"]  # Use the existing UniqueID
            read_name = f"{unique_id}_{column}"
            sequences_to_blat.append((read_name, sequence))

# Write the sequences to a FASTA file
with open(fasta_output_file, "w") as fasta_file:
    for read_name, sequence in sequences_to_blat:
        fasta_file.write(f">{read_name}\n{sequence}\n")

print(f"FASTA file for BLAT input created and saved to: {fasta_output_file}")

In [ ]:
%%writefile /nfs/users/nfs_a/ah39/Bash_scripts/BLAT_14.11.24.sh
#!/bin/bash
#BSUB -J blat_job
#BSUB -o /lustre/scratch126/casm/team274sb/ah39/allele_integrator/logs_errors/blat_%J.out
#BSUB -e /lustre/scratch126/casm/team274sb/ah39/allele_integrator/logs_errors/blat_%J.err
#BSUB -n 2
#BSUB -M 7000
#BSUB -R 'select[mem>7000] rusage[mem=7000]'
#BSUB -q normal

# Load conda environment
echo "Loading conda environment..."
source /software/cellgen/team274/miniconda3/etc/profile.d/conda.sh
conda activate blat_env

# Define paths
ref_genome="/nfs/srpipe_references/downloaded_from_10X/refdata-gex-GRCh38-2020-A/fasta/genome.fa"
fasta_input="/lustre/scratch126/casm/team274sb/ah39/allele_integrator/fusion_outputs/prepared_blat_input.fasta"
output_dir="/lustre/scratch126/casm/team274sb/ah39/allele_integrator/fusion_outputs"
blat_output="${output_dir}/blat_results.psl"

# Run BLAT with UCSC web tool parameters
echo "Starting BLAT with UCSC web-based parameters..."
blat $ref_genome $fasta_input $blat_output -stepSize=5 -repMatch=2253 -minScore=20 -minIdentity=0 -out=psl

echo "BLAT finished. Output saved to: $blat_output"

In [11]:
import pysam
import re
import os
import csv
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

def extract_soft_clipped_reads_with_barcodes(bamfile_path, chrom, start, end, min_softclip=20, bam_prefix="BAM", patient_id="Sample"):
    soft_clipped_reads = []
    cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')

    with pysam.AlignmentFile(bamfile_path, "rb") as bamfile:
        for read in bamfile.fetch(chrom, start, end):
            if read.is_unmapped:
                continue

            # Cell Barcode
            cell_barcode = dict(read.get_tags()).get("CB") or dict(read.get_tags()).get("CR")
            cell_barcode = cell_barcode.replace("-1-1", "-1") if cell_barcode else "Unknown"
            cell_id = f"{bam_prefix}::{cell_barcode}" if cell_barcode != "Unknown" else "Unknown"

            strand = "-" if read.is_reverse else "+"
            cigar_tuples = read.cigartuples
            cigar_string = read.cigarstring or ""
            read_sequence = read.query_sequence

            if not cigar_tuples or not read_sequence:
                continue

            # Determine if soft-clipping exists and meets threshold
            soft_clipped_seq = None
            soft_clipped_start = ""
            soft_clipped_end = ""
            soft_clip_start_pos = None
            soft_clip_end_pos = None

            parsed_cigar = cigar_pattern.findall(cigar_string)
            current_pos = read.reference_start
            for i, (length, op) in enumerate(parsed_cigar):
                length = int(length)
                if op == "S":
                    if i == 0 and length >= min_softclip:
                        soft_clipped_start = read_sequence[:length]
                        soft_clip_start_pos = current_pos - length
                    elif i == len(parsed_cigar) - 1 and length >= min_softclip:
                        soft_clipped_end = read_sequence[-length:]
                        soft_clip_end_pos = current_pos
                elif op in ("M", "D", "N", "="):
                    current_pos += length

            if soft_clipped_start or soft_clipped_end:
                aligned_start = read.reference_start
                aligned_end = read.reference_end
                soft_clipped_reads.append({
                    "PatientID": patient_id,
                    "BAM": bamfile_path,
                    "ReadID": read.query_name,
                    "Chromosome": chrom,
                    "Start": aligned_start,
                    "End": aligned_end,
                    "Strand": strand,
                    "CellBarcode": cell_barcode,
                    "CellID": cell_id,
                    "SoftClippedStart": soft_clipped_start,
                    "SoftClippedEnd": soft_clipped_end,
                    "SoftClipStartPos": soft_clip_start_pos,
                    "SoftClipEndPos": soft_clip_end_pos
                })

    return soft_clipped_reads

def save_soft_clipped_reads_to_csv(soft_clipped_reads, output_path):
    fieldnames = [
        "PatientID", "BAM", "ReadID", "Chromosome", "Start", "End", "Strand",
        "CellBarcode", "CellID", "SoftClippedStart", "SoftClippedEnd", 
        "SoftClipStartPos", "SoftClipEndPos"
    ]
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in soft_clipped_reads:
            writer.writerow(row)

def save_soft_clips_to_fasta(soft_clipped_reads, fasta_path):
    records = []
    for i, read in enumerate(soft_clipped_reads):
        for side, seq in [('start', read['SoftClippedStart']), ('end', read['SoftClippedEnd'])]:
            if seq:
                record_id = f"{read['ReadID']}|CG_SB_NB1444954:{read['CellBarcode']}|{side}"
                records.append(SeqRecord(Seq(seq), id=record_id, description=""))
    SeqIO.write(records, fasta_path, "fasta")

            
if __name__ == "__main__":
    bamfile_path = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/possorted_genome_bam51.bam"
    chromosome = "19"
    start = 42295000
    end = 42295200
    output_csv = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.csv"

    results = extract_soft_clipped_reads_with_barcodes(bamfile_path, chromosome, start, end, min_softclip=10, bam_prefix="Run1", patient_id="P01")
    save_soft_clipped_reads_to_csv(results, output_csv)
    print(f"Extracted {len(results)} soft-clipped reads. Output saved to {output_csv}.")
    
    fasta_output = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.fasta"
    save_soft_clips_to_fasta(results, fasta_output)
    print(f"FASTA file saved to {fasta_output}.")


Extracted 9 soft-clipped reads. Output saved to /lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.csv.
FASTA file saved to /lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.fasta.


In [ ]:
import subprocess
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

def save_soft_clips_to_fasta(soft_clipped_reads, fasta_path):
    records = []
    for i, read in enumerate(soft_clipped_reads):
        for side, seq in [('start', read['SoftClippedStart']), ('end', read['SoftClippedEnd'])]:
            if seq:
                record_id = f"{read['ReadID']}|{read['CellBarcode']}|{side}"
                records.append(SeqRecord(Seq(seq), id=record_id, description=""))
    SeqIO.write(records, fasta_path, "fasta")
